# Kanji to art

Given a Kanji (漢字), convert it into a text art rendition of itself.


## Example

Input: "字"

Output: 
```
　　　　　　　　　字　　　　　　　　　　
　　　　　　　　　字字　　　　　　　　　
　　字字字字字字字字字字字字字字字字字　
　　字　　　　　　　　　　　　　　字　　
　字字　　　　　　　　　　　　　字　　　
　字　　字字字字字字字字字字字　　　　　
　　　　　　　　　　　　　字字　　　　　
　　　　　　　　　　　字字　　　　　　　
　　　　　　　　　字字　　　　　　　　　
　　　　　　　　　字字　　　　　　　　　
　　　　　　　　　字字　　　　　　字字　
　字字字字字字字字字字字字字字字字字字　
　　　　　　　　　字　　　　　　　　　　
　　　　　　　　　字　　　　　　　　　　
　　　　　　　　　字　　　　　　　　　　
　　　　　　　　　字　　　　　　　　　　
　　　　　　字　　字　　　　　　　　　　
　　　　　　　　字字　　　　　　　　　　
```

## Algorithm


Input: `input_text`, a single kanji / 漢字 

Algorithm:
* create empty $N \times N$ pixel image $I$, i.e. $I \in \{1, 2, \dots, 255 \}^{N \times N}$
* print `input_text` onto $I$
* Threshold each image at 127 (note: this is mostly for convenience)
* Divide $I$ into chunks of size $M \times M$ (where $N$ is divisible by $M$)
* For each chunk: if more than 30% of a chunk is filled, replace it with `input_text`, empty space otherwise

Output: the art (string)

### Import modules

In [ ]:
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image, ImageDraw, ImageFont

### Initialise Kanji and parameters

To improve quality, increase `img_size`, and/or decrease `chunk_size`.

In [ ]:
input_text = "字" # kanji to turn into art

img_size = 500 # size of image (affects the resolution of the art)
chunk_size = 25  # size of each chunk (determines art resolution & final size of the art)
image = Image.new('L', (img_size, img_size)) # 'L' means greyscale

# note: can't use cv2.puttext because it doesn't support Kanji

draw = ImageDraw.Draw(image) # initialise drawing
font_text = ImageFont.truetype(font='mingliu.ttc', size=img_size, encoding='utf-8') # set font / font size / encoding
draw.text( (0, 0), input_text, fill=(255), font=font_text ) # coordinates (xy) ; fill = colour


### Visualise how the text art is created

In [ ]:
# show what the chunked boundaries will be (comment out if not debugging)
chunk_img = np.array(image.copy())
chunk_img = cv.cvtColor(chunk_img, cv.COLOR_GRAY2RGB) # convert to RGB image
for (i, j) in np.ndindex(img_size // chunk_size, img_size // chunk_size): 
    cv.line(chunk_img, (chunk_size*i, 0), (chunk_size*i, image.width), (0, 255, 0), 2) # draw vertical lines
    cv.line(chunk_img, (0, chunk_size*j), (image.height, chunk_size*j), (0, 255, 0), 2) # draw horizontal lines
_ = plt.imshow(chunk_img)

### Generate text art

In [ ]:
array = np.array(image, dtype=np.uint8)  # convert to np array for chunking
ret, array = cv.threshold(array, 127, 255, cv.THRESH_BINARY) # threshold to make computation easier
# _ = plt.imshow(array, cmap='gray', vmin=0, vmax=255) # for debugging

array[ array == 255 ] = 1 # makes computing easier later on
# chunk the image
chunked = np.array([array[i*chunk_size:(i+1)*chunk_size, j*chunk_size:(j+1)*chunk_size] for (i, j) in np.ndindex(img_size // chunk_size, img_size // chunk_size)])
# sum each chunk
chunked_summed = chunked.sum(axis=(1,2))
# determine if we should output the input_text or an empty space
threshold = int((chunked.shape[1] * chunked.shape[2]) * 0.3) # 0.3 is arbitrary
chunked_summed = np.where(chunked_summed > threshold, 1, 0)
chunked_summed = np.array(np.split(chunked_summed, img_size // chunk_size))


### Output text art

In [ ]:
final = "" # output string
for i, j in np.ndindex(chunked_summed.shape):
    final += input_text if chunked_summed[i,j] else "　"
    if j == chunked_summed.shape[1] - 1: final += "\n" # next line

print(final)
